<h2 style="font-family: 'Segoe UI', Arial, sans-serif; background-color: #f0f8ff; color: #004085; padding: 12px 15px; margin-top: 30px; border-left: 5px solid #007bff; border-radius: 4px; font-weight: 600;">
  LLM is worth trying
</h2>

**The actual score of the single large language model is 0.97138**

More recently, the emergence of large language models (LLMs) have demonstrated remarkable capabilities, achieving state-of-the-art results across a variety of tasks with fine-tuning.

**How does the large language model perform on tabular data?**

<h2 style="font-family: 'Segoe UI', Arial, sans-serif; background-color: #f0f8ff; color: #004085; padding: 12px 15px; margin-top: 30px; border-left: 5px solid #007bff; border-radius: 4px; font-weight: 600;">
  1.Data process
</h2>

 **1.1train eval data split**
 
Randomly shuffle the data, and take 650,000 rows of data as the training set, and 100,000 rows of data as the validation set.

 **1.2contruct prompt**

 Example:

【Category Definition】
- 1:client will subscribe
- 0:client will not subscribe
【Example 1】
- Input:age:43;job:technician;marital:married;education:secondary;default:no;balance:-274;housing_loan:yes;personal_loan:no;contact:cellular;day:26;month:aug;duration:160;campaign:2;pdays:-1;previous:0;poutcome:unknown
- Output:0
【Example 2】
- Input:age:27;job:management;marital:single;education:tertiary;default:no;balance:376;housing_loan:no;personal_loan:no;contact:cellular;day:3;month:dec;duration:641;campaign:2;pdays:205;previous:1;poutcome:failure
- Output:1
【Client Data to Predict】
- age:32;job:technician;marital:married;education:tertiary;default:no;balance:3716;housing_loan:yes;personal_loan:no;contact:cellular;day:19;month:nov;duration:167;campaign:1;pdays:-1;previous:0;poutcome:unknown
【Output Requirements】
- ONLY output "0" or "1" with NO additional text
- DO NOT explain or analyze
【Result】
- Output:

 **1.3contruct conversations**

 Example:

In [2]:
{
      "conversations": [
         {
            "value": """【Task】
You are an expert in bank marketing, Your goal is to predict whether a client will subscribe(0 or 1) to a bank term deposit based on client data.
【Category Definition】
- 1:client will subscribe
- 0:client will not subscribe
【Example 1】
- Input:age:43;job:technician;marital:married;education:secondary;default:no;balance:-274;housing_loan:yes;personal_loan:no;contact:cellular;day:26;month:aug;duration:160;campaign:2;pdays:-1;previous:0;poutcome:unknown
- Output:0
【Example 2】
- Input:age:27;job:management;marital:single;education:tertiary;default:no;balance:376;housing_loan:no;personal_loan:no;contact:cellular;day:3;month:dec;duration:641;campaign:2;pdays:205;previous:1;poutcome:failure
- Output:1
【Client Data to Predict】
- age:32;job:technician;marital:married;education:tertiary;default:no;balance:3716;housing_loan:yes;personal_loan:no;contact:cellular;day:19;month:nov;duration:167;campaign:1;pdays:-1;previous:0;poutcome:unknown
【Output Requirements】
- ONLY output "0" or "1" with NO additional text
- DO NOT explain or analyze
【Result】
- Output:""",
            "from": "user"
         },
         {
            "value": "0",
            "from": "gpt"
         }
      ]
   }

{'conversations': [{'value': '【Task】\nYou are an expert in bank marketing, Your goal is to predict whether a client will subscribe(0 or 1) to a bank term deposit based on client data.\n【Category Definition】\n- 1:client will subscribe\n- 0:client will not subscribe\n【Example 1】\n- Input:age:43;job:technician;marital:married;education:secondary;default:no;balance:-274;housing_loan:yes;personal_loan:no;contact:cellular;day:26;month:aug;duration:160;campaign:2;pdays:-1;previous:0;poutcome:unknown\n- Output:0\n【Example 2】\n- Input:age:27;job:management;marital:single;education:tertiary;default:no;balance:376;housing_loan:no;personal_loan:no;contact:cellular;day:3;month:dec;duration:641;campaign:2;pdays:205;previous:1;poutcome:failure\n- Output:1\n【Client Data to Predict】\n- age:32;job:technician;marital:married;education:tertiary;default:no;balance:3716;housing_loan:yes;personal_loan:no;contact:cellular;day:19;month:nov;duration:167;campaign:1;pdays:-1;previous:0;poutcome:unknown\n【Output R

<h2 style="font-family: 'Segoe UI', Arial, sans-serif; background-color: #f0f8ff; color: #004085; padding: 12px 15px; margin-top: 30px; border-left: 5px solid #007bff; border-radius: 4px; font-weight: 600;">
  2.fine tuning
</h2>

**I used the LLM(llama3.1-8b-instruct version) and the llama-factory fine-tuning framework. The training parameters are as follows.**

model_name_or_path: "/data/jupyter/dev/lyf/llms/LLM-Research/Meta-Llama-3.1-8B-Instruct"
trust_remote_code: True


###
stage: sft
do_train: True
finetuning_type: lora
lora_rank: 32
lora_target: "q_proj,v_proj"


deepspeed: "examples/deepspeed/ds_z2_config.json
### dataset
dataset: kaggle_clf_train_data
template: llama3
cutoff_len: 360
overwrite_cache: true
preprocessing_num_workers: 16
dataloader_num_workers: 0

### output
output_dir: "/data/jupyter/dev/lyf/LLaMA-Factory/examples/save/adapter_llama31_kaggle_v5"
logging_steps: 100
save_steps: 500
plot_loss: true
overwrite_output_dir: true
save_only_model: false
report_to: none  

### train
per_device_train_batch_size: 16
gradient_accumulation_steps: 4
learning_rate: 3.0e-5
num_train_epochs: 12.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000
resume_from_checkpoint: null

### eval
eval_dataset: kaggle_clf_val_data
per_device_eval_batch_size: 32
eval_strategy: steps
eval_steps: 500

<h2 style="font-family: 'Segoe UI', Arial, sans-serif; background-color: #f0f8ff; color: #004085; padding: 12px 15px; margin-top: 30px; border-left: 5px solid #007bff; border-radius: 4px; font-weight: 600;">
  3.merge model
</h2>

**Select the checkpoint with the lowest validation loss from the list, and the log is as follows**

{"current_steps": 18800, "total_steps": 30480, "loss": 0.0588, "lr": 1.1535747525971477e-05, "epoch": 7.401693413409471, "percentage": 61.68, "elapsed_time": "1 day, 22:25:54", "remaining_time": "1 day, 4:50:48"}
{"current_steps": 18900, "total_steps": 30480, "loss": 0.0602, "lr": 1.1368838069625684e-05, "epoch": 7.441075120606478, "percentage": 62.01, "elapsed_time": "1 day, 22:37:49", "remaining_time": "1 day, 4:34:13"}
{"current_steps": 19000, "total_steps": 30480, "loss": 0.0616, "lr": 1.1202404853062242e-05, "epoch": 7.4804568278034855, "percentage": 62.34, "elapsed_time": "1 day, 22:49:45", "remaining_time": "1 day, 4:17:41"}
{"current_steps": 19000, "total_steps": 30480, "eval_loss": 0.06775250285863876, "epoch": 7.4804568278034855, "percentage": 62.34, "elapsed_time": "1 day, 23:04:20", "remaining_time": "1 day, 4:26:29"}
{"current_steps": 19100, "total_steps": 30480, "loss": 0.0612, "lr": 1.1036469704588548e-05, "epoch": 7.519838535000492, "percentage": 62.66, "elapsed_time": "1 day, 23:16:25", "remaining_time": "1 day, 4:09:58"}
{"current_steps": 19200, "total_steps": 30480, "loss": 0.0596, "lr": 1.0871054387188615e-05, "epoch": 7.559220242197499, "percentage": 62.99, "elapsed_time": "1 day, 23:28:20", "remaining_time": "1 day, 3:53:24"}
{"current_steps": 19300, "total_steps": 30480, "loss": 0.0566, "lr": 1.070618059566873e-05, "epoch": 7.5986019493945065, "percentage": 63.32, "elapsed_time": "1 day, 23:40:16", "remaining_time": "1 day, 3:36:53"}
{"current_steps": 19400, "total_steps": 30480, "loss": 0.0618, "lr": 1.0541869953812177e-05, "epoch": 7.637983656591513, "percentage": 63.65, "elapsed_time": "1 day, 23:52:11", "remaining_time": "1 day, 3:20:24"}
{"current_steps": 19500, "total_steps": 30480, "loss": 0.0603, "lr": 1.0378144011543138e-05, "epoch": 7.67736536378852, "percentage": 63.98, "elapsed_time": "2 days, 0:04:07", "remaining_time": "1 day, 3:03:59"}
{"current_steps": 19500, "total_steps": 30480, "eval_loss": 0.06741023063659668, "epoch": 7.67736536378852, "percentage": 63.98, "elapsed_time": "2 days, 0:18:42", "remaining_time": "1 day, 3:12:11"}
{"current_steps": 19600, "total_steps": 30480, "loss": 0.0594, "lr": 1.0215024242100375e-05, "epoch": 7.7167470709855275, "percentage": 64.3, "elapsed_time": "2 days, 0:30:47", "remaining_time": "1 day, 2:55:47"}
{"current_steps": 19700, "total_steps": 30480, "loss": 0.059, "lr": 1.0052532039220934e-05, "epoch": 7.756128778182534, "percentage": 64.63, "elapsed_time": "2 days, 0:42:43", "remaining_time": "1 day, 2:39:20"}
{"current_steps": 19800, "total_steps": 30480, "loss": 0.0597, "lr": 9.890688714334268e-06, "epoch": 7.795510485379541, "percentage": 64.96, "elapsed_time": "2 days, 0:54:38", "remaining_time": "1 day, 2:22:55"}
{"current_steps": 19900, "total_steps": 30480, "loss": 0.0606, "lr": 9.729515493767187e-06, "epoch": 7.8348921925765485, "percentage": 65.29, "elapsed_time": "2 days, 1:06:34", "remaining_time": "1 day, 2:06:34"}
{"current_steps": 20000, "total_steps": 30480, "loss": 0.0602, "lr": 9.56903351595993e-06, "epoch": 7.874273899773555, "percentage": 65.62, "elapsed_time": "2 days, 1:18:30", "remaining_time": "1 day, 1:50:15"}
{"current_steps": 20000, "total_steps": 30480, "eval_loss": 0.06710181385278702, "epoch": 7.874273899773555, "percentage": 65.62, "elapsed_time": "2 days, 1:33:06", "remaining_time": "1 day, 1:57:54"}
{"current_steps": 20100, "total_steps": 30480, "loss": 0.059, "lr": 9.409263828693805e-06, "epoch": 7.913655606970562, "percentage": 65.94, "elapsed_time": "2 days, 1:45:11", "remaining_time": "1 day, 1:41:36"}
{"current_steps": 20200, "total_steps": 30480, "loss": 0.059, "lr": 9.250227386330683e-06, "epoch": 7.953037314167569, "percentage": 66.27, "elapsed_time": "2 days, 1:57:07", "remaining_time": "1 day, 1:25:16"}
{"current_steps": 20300, "total_steps": 30480, "loss": 0.0595, "lr": 9.091945047064749e-06, "epoch": 7.992419021364576, "percentage": 66.6, "elapsed_time": "2 days, 2:09:02", "remaining_time": "1 day, 1:08:58"}
{"current_steps": 20400, "total_steps": 30480, "loss": 0.0573, "lr": 8.934437570186905e-06, "epoch": 8.031505365757605, "percentage": 66.93, "elapsed_time": "2 days, 2:20:53", "remaining_time": "1 day, 0:52:40"}
{"current_steps": 20500, "total_steps": 30480, "loss": 0.0557, "lr": 8.777725613362087e-06, "epoch": 8.070887072954612, "percentage": 67.26, "elapsed_time": "2 days, 2:32:49", "remaining_time": "1 day, 0:36:27"}
{"current_steps": 20500, "total_steps": 30480, "eval_loss": 0.06713543087244034, "epoch": 8.070887072954612, "percentage": 67.26, "elapsed_time": "2 days, 2:47:23", "remaining_time": "1 day, 0:43:33"}
{"current_steps": 20600, "total_steps": 30480, "loss": 0.0584, "lr": 8.621829729919962e-06, "epoch": 8.11026878015162, "percentage": 67.59, "elapsed_time": "2 days, 2:59:29", "remaining_time": "1 day, 0:27:22"}
{"current_steps": 20700, "total_steps": 30480, "loss": 0.0576, "lr": 8.466770366159281e-06, "epoch": 8.149650487348627, "percentage": 67.91, "elapsed_time": "2 days, 3:11:25", "remaining_time": "1 day, 0:11:08"}
{"current_steps": 20800, "total_steps": 30480, "loss": 0.0581, "lr": 8.31256785866623e-06, "epoch": 8.189032194545634, "percentage": 68.24, "elapsed_time": "2 days, 3:23:21", "remaining_time": "23:54:57"}
{"current_steps": 20900, "total_steps": 30480, "loss": 0.0593, "lr": 8.159242431647295e-06, "epoch": 8.228413901742641, "percentage": 68.57, "elapsed_time": "2 days, 3:35:18", "remaining_time": "23:38:48"}
{"current_steps": 21000, "total_steps": 30480, "loss": 0.0601, "lr": 8.00681419427673e-06, "epoch": 8.267795608939647, "percentage": 68.9, "elapsed_time": "2 days, 3:47:14", "remaining_time": "23:22:41"}
{"current_steps": 21000, "total_steps": 30480, "eval_loss": 0.06711243093013763, "epoch": 8.267795608939647, "percentage": 68.9, "elapsed_time": "2 days, 4:01:48", "remaining_time": "23:29:16"}
{"current_steps": 21100, "total_steps": 30480, "loss": 0.0604, "lr": 7.85530313805917e-06, "epoch": 8.307177316136654, "percentage": 69.23, "elapsed_time": "2 days, 4:13:54", "remaining_time": "23:13:10"}
{"current_steps": 21200, "total_steps": 30480, "loss": 0.0593, "lr": 7.7047291342077e-06, "epoch": 8.346559023333661, "percentage": 69.55, "elapsed_time": "2 days, 4:25:50", "remaining_time": "22:57:02"}
{"current_steps": 21300, "total_steps": 30480, "loss": 0.0576, "lr": 7.555111931037643e-06, "epoch": 8.385940730530669, "percentage": 69.88, "elapsed_time": "2 days, 4:37:46", "remaining_time": "22:40:57"}
{"current_steps": 21400, "total_steps": 30480, "loss": 0.0566, "lr": 7.406471151376532e-06, "epoch": 8.425322437727676, "percentage": 70.21, "elapsed_time": "2 days, 4:49:42", "remaining_time": "22:24:54"}
{"current_steps": 21500, "total_steps": 30480, "loss": 0.0568, "lr": 7.258826289990472e-06, "epoch": 8.464704144924683, "percentage": 70.54, "elapsed_time": "2 days, 5:01:38", "remaining_time": "22:08:53"}
{"current_steps": 21500, "total_steps": 30480, "eval_loss": 0.06746434420347214, "epoch": 8.464704144924683, "percentage": 70.54, "elapsed_time": "2 days, 5:16:13", "remaining_time": "22:14:58"}
{"current_steps": 21600, "total_steps": 30480, "loss": 0.0571, "lr": 7.112196711027363e-06, "epoch": 8.504085852121689, "percentage": 70.87, "elapsed_time": "2 days, 5:28:18", "remaining_time": "21:58:58"}
{"current_steps": 21700, "total_steps": 30480, "loss": 0.0568, "lr": 6.966601645477213e-06, "epoch": 8.543467559318696, "percentage": 71.19, "elapsed_time": "2 days, 5:40:14", "remaining_time": "21:42:56"}
{"current_steps": 21800, "total_steps": 30480, "loss": 0.057, "lr": 6.822060188649904e-06, "epoch": 8.582849266515703, "percentage": 71.52, "elapsed_time": "2 days, 5:52:10", "remaining_time": "21:26:56"}
{"current_steps": 21900, "total_steps": 30480, "loss": 0.0566, "lr": 6.678591297670801e-06, "epoch": 8.62223097371271, "percentage": 71.85, "elapsed_time": "2 days, 6:04:06", "remaining_time": "21:10:58"}
{"current_steps": 22000, "total_steps": 30480, "loss": 0.0577, "lr": 6.536213788994454e-06, "epoch": 8.661612680909718, "percentage": 72.18, "elapsed_time": "2 days, 6:16:02", "remaining_time": "20:55:03"}
{"current_steps": 22000, "total_steps": 30480, "eval_loss": 0.06738024204969406, "epoch": 8.661612680909718, "percentage": 72.18, "elapsed_time": "2 days, 6:30:37", "remaining_time": "21:00:40"}
{"current_steps": 22100, "total_steps": 30480, "loss": 0.0574, "lr": 6.394946335936752e-06, "epoch": 8.700994388106725, "percentage": 72.51, "elapsed_time": "2 days, 6:42:42", "remaining_time": "20:44:45"}
{"current_steps": 22200, "total_steps": 30480, "loss": 0.0569, "lr": 6.254807466225834e-06, "epoch": 8.74037609530373, "percentage": 72.83, "elapsed_time": "2 days, 6:54:38", "remaining_time": "20:28:48"}
{"current_steps": 22300, "total_steps": 30480, "loss": 0.0581, "lr": 6.1158155595721235e-06, "epoch": 8.779757802500738, "percentage": 73.16, "elapsed_time": "2 days, 7:06:34", "remaining_time": "20:12:54"}
{"current_steps": 22400, "total_steps": 30480, "loss": 0.058, "lr": 5.97798884525779e-06, "epoch": 8.819139509697745, "percentage": 73.49, "elapsed_time": "2 days, 7:18:30", "remaining_time": "19:57:02"}
{"current_steps": 22500, "total_steps": 30480, "loss": 0.057, "lr": 5.841345399745858e-06, "epoch": 8.858521216894752, "percentage": 73.82, "elapsed_time": "2 days, 7:30:26", "remaining_time": "19:41:11"}
{"current_steps": 22500, "total_steps": 30480, "eval_loss": 0.06743192672729492, "epoch": 8.858521216894752, "percentage": 73.82, "elapsed_time": "2 days, 7:45:01", "remaining_time": "19:46:22"}

<h2 style="font-family: 'Segoe UI', Arial, sans-serif; background-color: #f0f8ff; color: #004085; padding: 12px 15px; margin-top: 30px; border-left: 5px solid #007bff; border-radius: 4px; font-weight: 600;">
  4.inference
</h2>

**4.1 load model**

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# 1. 加载模型与分词器
model_name = "/data/jupyter/dev/lyf/LLaMA-Factory/examples/save/kaggle_llama31_sft_v5"  # 可选其他版本如 0.5B/1.5B/14B
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda:1",
    torch_dtype=torch.bfloat16,  # 优化显存，精度足够
    trust_remote_code=True
)

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/data/jupyter/dev/lyf/LLaMA-Factory/examples/save/kaggle_llama31_sft_v5'. Use `repo_type` argument if needed.

**4.2 Output probability**

In [8]:
res_data = []
prompt = """【Task】
You are an expert in bank marketing, Your goal is to predict whether a client will subscribe(0 or 1) to a bank term deposit based on client data.
【Category Definition】
- 1:client will subscribe
- 0:client will not subscribe
【Example 1】
- Input:age:43;job:technician;marital:married;education:secondary;default:no;balance:-274;housing_loan:yes;personal_loan:no;contact:cellular;day:26;month:aug;duration:160;campaign:2;pdays:-1;previous:0;poutcome:unknown
- Output:0
【Example 2】
- Input:age:27;job:management;marital:single;education:tertiary;default:no;balance:376;housing_loan:no;personal_loan:no;contact:cellular;day:3;month:dec;duration:641;campaign:2;pdays:205;previous:1;poutcome:failure
- Output:1
【Client Data to Predict】
- {0}
【Output Requirements】
- ONLY output "0" or "1" with NO additional text
- DO NOT explain or analyze
【Result】
- Output:"""
for index, row in test_data.iterrows():
    id_no = row['id']
    temp_str = "age:" + str(row['age']) + ";" + "job:" + row['job'] + ";" + "marital:" + row['marital'] + ";" + "education:" + row['education'] +";" + "default:" + row['default'] + ";" + "balance:" + str(row['balance']) + ";" + "housing_loan:" + row['housing'] + ";" + "personal_loan:" + row['loan'] +";"+ "contact:" + row['contact']  + ";"+ "day:" + str(row['day']) + ";" + "month:" + row['month']+ ";" + "duration:" + str(row['duration']) + ";" + "campaign:" + str(row['campaign']) + ";" + "pdays:" + str(row['pdays']) + ";" + "previous:" + str(row['previous'])+ ";" + "poutcome:" + row['poutcome']
    input_text = prompt.format(temp_str)
#     print(input_text)
#     input_text = "Your goal is to predict whether a client will subscribe to a bank term deposit.The following is the relevant information of the data.\nage:26,job:technician,marital:married,education:secondary,default:no,balance:889,housing:yes,loan:no,contact:cellular,day:3,month:feb,duration:902,campaign:1,pdays:-1,previous:0,poutcome:unknown\nOnly 0 or 1 needs to be output"
    if index%100==0:
        print(index)
    
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(inputs.input_ids,do_sample=False, max_new_tokens=1,num_beams=2,early_stopping=True)

    #输出一个字的结果
    final_output = tokenizer.decode(outputs[0][-1], skip_special_tokens=True)
    token_id_1 = tokenizer.convert_tokens_to_ids("1")
    token_id_0 = tokenizer.convert_tokens_to_ids("0")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[:, -1, [token_id_1, token_id_0]]
        print(logits)
        probs = torch.softmax(logits, dim=-1)  # 概率分布 [batch_size, vocab_size]
        probs_05 = torch.softmax(logits/0.5, dim=-1)[0].tolist()[0]
        probs_08 = torch.softmax(logits/0.8, dim=-1)[0].tolist()[0]
        probs_12 = torch.softmax(logits/1.2, dim=-1)[0].tolist()[0]
    prob_1, prob_0 = probs[0].tolist()
    res_data.append([id_no, prob_1,probs_05,probs_08,probs_12])
    print(f"id_no {id_no},生成结果: {final_output} |概率: 1={prob_1:.6f}, 1_05={probs_05:.6f},1_08={probs_08:.6f},1_12={probs_12:.6f}")


NameError: name 'test_data' is not defined

In [ ]:
import pandas as pd
sub = pd.read_csv('/kaggle/input/llama31-single-res/submission (2).csv')
sub[0:10]

In [ ]:
sub.to_csv('/kaggle/working/submission.csv',index=False)